In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.65


In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config(path=".")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

UserErrorException: We could not find config.json in: D:\ or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.

In [18]:
from azureml.core.compute import ComputeTarget, BatchAiCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "LEGOVISION"

try:
    compute_target_one = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = BatchAiCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                                autoscale_enabled=True,
                                                                cluster_min_nodes=0, 
                                                                cluster_max_nodes=2)

    # create the cluster
    compute_target_one = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target_one.wait_for_completion(show_output=True)

    # Use the 'status' property to get a detailed status for the current cluster. 
    print(compute_target_one.status.serialize())

Creating a new compute target...
Creating
succeeded
BatchAI wait for completion finished
Minimum number of nodes requested have been provisioned
{'allocationState': 'steady', 'allocationStateTransitionTime': '2018-11-23T10:23:15.224000+00:00', 'creationTime': '2018-11-23T10:23:04.916000+00:00', 'currentNodeCount': 0, 'errors': None, 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'succeeded', 'provisioningStateTransitionTime': '2018-11-23T10:23:14.584000+00:00', 'scaleSettings': {'manual': None, 'autoScale': {'maximumNodeCount': 2, 'minimumNodeCount': 0, 'initialNodeCount': 0}}, 'vmPriority': 'dedicated', 'vmSize': 'STANDARD_NC6'}


In [9]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

AzureFile legovision1529597820 azureml-filestore-f0102bff-6d0a-47ad-b50c-4709fd2f206a


In [10]:
annotated_data = ds.path('Object Detection')
print(annotated_data)

$AZUREML_DATAREFERENCE_c6ecf1033ae04686b31eb858f56795a1


In [36]:
import shutil
project_folder = './faster-rcnn-aml'
shutil.rmtree(project_folder)
shutil.copytree('Faster R-CNN', project_folder)

'./faster-rcnn-aml'

In [12]:
from azureml.core import Experiment

experiment_name = 'Faster_R-CNN'
experiment = Experiment(ws, name=experiment_name)

In [38]:
from azureml.train.estimator import *

script_params = {
    '--base_directory': annotated_data.as_mount(),
    '--data_set_name': "LEGO Vision 500s"
}

estimator = Estimator(source_directory=project_folder,
                      compute_target=compute_target_one,
                      entry_script='run_faster_rcnn.py',
                      script_params=script_params,
                      pip_packages=['cntk-gpu==2.6'],
                      custom_docker_base_image='microsoft/mmlspark:gpu',
                      use_gpu=True)

In [40]:
run = experiment.submit(estimator)

In [42]:
from azureml.train.widgets import RunDetails
RunDetails(run).show()

_UserRun(widget_settings={'childWidgetDisplay': 'popup'})

MSI: Failed to retrieve a token from 'http://localhost:25198/nb/api/nbsvc/oauth2/token' with an error of 'HTTPConnectionPool(host='localhost', port=25198): Max retries exceeded with url: /nb/api/nbsvc/oauth2/token (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f813dd39978>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))'. This could be caused by the MSI extension not yet fullly provisioned.
